# Imports

In [1]:
! pip install pyreadr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 10.7 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadr
from google.colab import drive
from typing import List

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# Lectura de los datos

In [4]:
df = pyreadr.read_r('/content/drive/MyDrive/TFM/Data/day_ahead2016_2020.RData')

In [5]:
df['day_ahead_supply_prices']['price'] = df['day_ahead_supply_prices'].apply(lambda row: row.iloc[2:].tolist(), axis=1)
df['day_ahead_supply_cumsum']['supply'] = df['day_ahead_supply_cumsum'].apply(lambda row: row.iloc[2:].tolist(), axis=1)

In [6]:
df['day_ahead_supply_prices'].head(2)

,hour,date,x1,x2,x3,x4,x5,x6,x7,x8,...,x690,x691,x692,x693,x694,x695,x696,x697,x698,price
0,1,01/01/2016,0.0,0.1,0.13,0.5,0.57,0.6,1.0,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.0, 0.1, 0.13, 0.5, 0.57, 0.6, 1.0, 1.1, 1.3..."
1,2,01/01/2016,0.0,0.1,0.13,0.5,0.57,0.6,1.0,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.0, 0.1, 0.13, 0.5, 0.57, 0.6, 1.0, 1.1, 1.1..."


## Juntar todo en un solo dataframe

In [7]:
dates = list(df['day_ahead_supply_prices']['date'].values)
hours = list(df['day_ahead_supply_prices']['hour'].values)
prices = list(df['day_ahead_supply_prices']['price'].values)
supplies = list(df['day_ahead_supply_cumsum']['supply'].values)

# Quitamos los nan
prices_copy = []
supplies_copy = []
for price, supply in zip(prices, supplies):
    prices_copy.append([p for p in price if p > -10**-6])
    supplies_copy.append([s for s in supply if s > -10**-6])

df = pd.DataFrame({'date': dates, 'hour': hours, 'price': prices_copy, 'supply': supplies_copy})

In [8]:
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df['year'] = df['date'].apply(lambda x: x.year)
df = df[df['year'] < 2020]
df = df[['date', 'hour', 'price', 'supply']]

In [9]:
df.head(2)

,date,hour,price,supply
0,2016-01-01,1,"[0.0, 0.1, 0.13, 0.5, 0.57, 0.6, 1.0, 1.1, 1.3...","[24390.2, 24750.2, 25290.2, 25533.8, 25537.600..."
1,2016-01-01,2,"[0.0, 0.1, 0.13, 0.5, 0.57, 0.6, 1.0, 1.1, 1.1...","[24209.4, 24569.4, 24749.4, 24971.800000000003..."


# Mallas

## Malla equiespaciada

Para cada día tendremos que predecir la cantidad desde el mínimo precio histórico (0.01) hasta el máximo histórico (180.3). Para ello se hará un grid de 0 a 180 con paso 1 para cada hora de cada día. La cantidad asignada será la más cercana al precio correspondiente.

In [11]:
def get_new_supply(
        price: List[float],
        supply: List[float]
    ) -> List[float]:

    """
    Funcion para obtener la oferta asociada a la malla equiespaciada para cada fila.

    Parameters
    ----------
    price      : Lista de precios originales.
    supply     : Lista de ofertas originales.

    Returns
    -------
    new_supply : Oferta asociada a la malla equiespaciada.
    """
    
    # Numeros enteros de 0 a 180
    numeros_enteros = np.arange(181)

    # Indice del elemento mas cercano en price a cada numero entero
    idx = [np.abs(price - n).argmin() for n in numeros_enteros]

    # Lista que los elementos correspondientes
    new_supply = [supply[i] for i in idx]

    return new_supply

In [12]:
new_supply = []

for p, s in zip(df['price'].values, df['supply'].values):
    new_s = get_new_supply(p, s)
    new_supply.append(new_s)

df['price_180'] = [list(range(181)) for _ in range(len(df))]
df['supply_180'] = new_supply

## Malla no equiespaciada

In [13]:
# Desviacion tipica para cada precio
supp = list(df['supply_180'].values)
supp_mat = np.matrix([np.array(s) for s in supp])
supp_std = list(np.array(supp_mat.std(axis=0))[0])

In [14]:
def f(x): # funcion de masa
    return supp_std[x] / sum(supp_std)

In [15]:
num_points = 60 # numero de puntos de la rejilla

cumulative_probs = np.cumsum(np.array([f(i) for i in range(181)])) # valores de la funcion de distribucion

quantiles = np.linspace(0, 1, num_points)
grid = np.interp(quantiles, cumulative_probs, np.arange(181)) # mallado no equiespaciado

# Cogemos los numeros enteros, ya que nos estabamos basando en el mallado equiespaciado
grid = [int(node) for node in np.round(grid)]

In [16]:
new_supply = []

for s in df['supply_180'].values:
    new_s = [s[node] for node in grid]
    new_supply.append(new_s)

df['price_' + str(num_points)] = [grid for _ in range(len(df))]
df['supply_' + str(num_points)] = new_supply

# Guardado del dataframe

In [22]:
df.head(2)

,date,hour,price,supply,price_180,supply_180,price_60,supply_60
0,2016-01-01,1,"[0.0, 0.1, 0.13, 0.5, 0.57, 0.6, 1.0, 1.1, 1.3...","[24390.2, 24750.2, 25290.2, 25533.8, 25537.600...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[24390.2, 26266.4, 26291.8, 27832.4, 28305.2, ...","[0, 4, 8, 12, 16, 20, 23, 27, 31, 34, 37, 41, ...","[24390.2, 28305.2, 30847.0, 31282.8, 31375.2, ..."
1,2016-01-01,2,"[0.0, 0.1, 0.13, 0.5, 0.57, 0.6, 1.0, 1.1, 1.1...","[24209.4, 24569.4, 24749.4, 24971.800000000003...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[24209.4, 25668.800000000003, 25777.0, 27521.0...","[0, 4, 8, 12, 16, 20, 23, 27, 31, 34, 37, 41, ...","[24209.4, 28047.4, 30280.2, 30712.4, 30790.0, ..."


In [ ]:
df.to_csv('/content/drive/MyDrive/TFM/Data/df.csv', index=False)